# Main Data

* 여러 데이터 원본을 합쳐 하나의 메인 데이터로 구축하는 과정

In [70]:
import re
import pandas as pd

from datetime import datetime

In [71]:
re_dust_df = pd.read_csv('../data/seoul_re_dust.csv', index_col=0)
fine_dust_df = pd.read_csv('../data/fine_dust_df.csv', index_col=0)
bus_stop_df = pd.read_csv('../data/bus_stop_dong.csv', index_col=0)
construct_df = pd.read_csv('../data/contsruct_info.csv')
bus_df = pd.read_csv('../data/bus_df.csv', index_col=0)
map_df = pd.read_csv('../data_preprocess/구코드.csv')
dong_map_df = pd.read_csv('../data_preprocess/행정동코드.csv')

## 매핑용 테이블 처리
* 현재 공릉1동 이런 식으로 들어가있는데, 재비산먼지의 경우 공릉동으로 합쳐져 들어가있음
* 따라서 dong2라는 컬럼으로 숫자를 제외한 동 추가해줌

In [72]:
dong_map_df['dong2'] = dong_map_df['H_DNG_NM'].apply(lambda x: x[:-1])
dong_map_df['dong2'] = dong_map_df['dong2'].apply(lambda x: re.sub(r'\d+', '', x) + '동')

## 재비산먼지 처리
* 주소로 들어가 있기 때문에 해당 주소에서 OO동 정보 추출

In [73]:
re_dust_df = re_dust_df.dropna()
re_dust_df['dong'] = re_dust_df['start'].apply(lambda x: re.findall(r'\S+동', x)[0] if re.findall(r'\S+동', x) else '')

In [74]:
# 조인용으로 date_cd라는 컬럼 새로 만들어둠
re_dust_df['date_cd'] = re_dust_df['date'].apply(lambda x: "".join(x.split('-')))

In [75]:
# 동 정보 없는 경우는 제외(도로명인 경우)
re_dust_df = re_dust_df[re_dust_df['dong']!='']

In [76]:
# 도로 단위 데이터이다 보니, 
# 동별로 합칠 때 평균 보다는 동별 차이를 더 키워주기 위하여 최댓값을 가져옴
main_df = re_dust_df.groupby(['date_cd', 'gu', 'dong'])[['temperature', 'humidity', 'avg_re_dust', ]].max().reset_index()

In [77]:
main_df.head()

,date_cd,gu,dong,temperature,humidity,avg_re_dust
0,20170307,강서구,가양동,4.0,35.0,16.0
1,20170307,강서구,내발산동,3.0,34.0,2.0
2,20170307,강서구,등촌동,4.0,35.0,5.0
3,20170307,강서구,방화동,10.0,42.0,27.0
4,20170307,강서구,염창동,5.0,39.0,30.0


## 버스 승하차 인원 처리

In [78]:
bus_df = pd.merge(bus_df, dong_map_df, left_on='ADMDONG_ID', right_on='H_SDNG_CD')

In [79]:
# 세부동별 데이터를 동별로 바꿔주다보니 sum()으로 해당 동의 일별 승하차 인원의 합 구함
bus_df_new = bus_df.groupby(['CRTR_DT', 'dong2'])['BUS_PSGR_CNT'].sum()
bus_df_new = bus_df_new.reset_index()

In [80]:
bus_df_new['CRTR_DT'] = bus_df_new['CRTR_DT'].astype(str)

## 버스정류장수 처리

In [81]:
bus_stop_df = pd.merge(bus_stop_df, dong_map_df, left_on='구', right_on='H_DNG_NM')

In [82]:
bus_stop_new_df = bus_stop_df.groupby('dong2')['빈도수'].sum().reset_index()

## 대기오염지표 처리
* 동단위 데이터가 없어 구 단위로 반영

In [83]:
fine_dust_df['MSRDT_DE'] = fine_dust_df['MSRDT_DE'].astype(str)

In [84]:
fine_dust_df.head()

,MSRDT_DE,MSRRGN_NM,MSRSTE_NM,PM10,PM25,O3,NO2,CO,SO2
0,20180503,도심권,중구,22,11,0.036,0.022,0.4,0.002
1,20180503,도심권,종로구,23,11,0.039,0.021,0.5,0.005
2,20180503,도심권,용산구,23,12,0.031,0.024,0.4,0.002
3,20180503,서북권,은평구,28,14,0.037,0.015,0.4,0.003
4,20180503,서북권,서대문구,30,14,0.038,0.017,0.6,0.002


## Table Join

In [85]:
# 재비산먼지 - 버스승하차
merged_df = pd.merge(main_df, bus_df_new, left_on=['date_cd', 'dong'], right_on=['CRTR_DT', 'dong2'])

In [86]:
merged_df.head()

,date_cd,gu,dong,temperature,humidity,avg_re_dust,CRTR_DT,dong2,BUS_PSGR_CNT
0,20220207,용산구,이촌동,1.0,36.0,46.0,20220207,이촌동,7290
1,20220207,중구,소공동,2.0,33.0,25.0,20220207,소공동,26013
2,20220208,송파구,자양동,6.0,27.0,36.0,20220208,자양동,23643
3,20220209,마포구,연남동,5.0,38.0,213.0,20220209,연남동,6089
4,20220209,서대문구,남가좌동,6.0,33.0,49.0,20220209,남가좌동,19893


In [87]:
# 재비산먼지 - 버스승하차 - 버스정류장수
merged_df2 = pd.merge(merged_df, bus_stop_new_df, left_on='dong', right_on='dong2')

In [88]:
merged_df2.head()

,date_cd,gu,dong,temperature,humidity,avg_re_dust,CRTR_DT,dong2_x,BUS_PSGR_CNT,dong2_y,빈도수
0,20220207,용산구,이촌동,1.0,36.0,46.0,20220207,이촌동,7290,이촌동,33
1,20220311,용산구,이촌동,11.0,63.0,13.0,20220311,이촌동,8118,이촌동,33
2,20220318,용산구,이촌동,9.0,67.0,20.0,20220318,이촌동,7655,이촌동,33
3,20220518,용산구,이촌동,26.0,46.0,139.0,20220518,이촌동,9342,이촌동,33
4,20220616,용산구,이촌동,22.0,90.0,28.0,20220616,이촌동,10009,이촌동,33


In [89]:
# 재비산먼지 - 버스승하차 - 버스정류장수 - 대기오염지표
merged_df3 = pd.merge(merged_df2, fine_dust_df, left_on=['date_cd', 'gu'], right_on=['MSRDT_DE', 'MSRSTE_NM'])

In [90]:
merged_df3.head()

,date_cd,gu,dong,temperature,humidity,avg_re_dust,CRTR_DT,dong2_x,BUS_PSGR_CNT,dong2_y,빈도수,MSRDT_DE,MSRRGN_NM,MSRSTE_NM,PM10,PM25,O3,NO2,CO,SO2
0,20220207,용산구,이촌동,1.0,36.0,46.0,20220207,이촌동,7290,이촌동,33,20220207,도심권,용산구,28,16,0.021,0.031,0.5,0.004
1,20220311,용산구,이촌동,11.0,63.0,13.0,20220311,이촌동,8118,이촌동,33,20220311,도심권,용산구,67,36,0.022,0.043,0.6,0.004
2,20220318,용산구,이촌동,9.0,67.0,20.0,20220318,이촌동,7655,이촌동,33,20220318,도심권,용산구,6,3,0.032,0.010,0.3,0.002
3,20220518,용산구,이촌동,26.0,46.0,139.0,20220518,이촌동,9342,이촌동,33,20220518,도심권,용산구,59,34,0.079,0.017,0.5,0.004
4,20220518,용산구,서빙고동,26.0,45.0,16.0,20220518,서빙고동,7282,서빙고동,24,20220518,도심권,용산구,59,34,0.079,0.017,0.5,0.004


## 공사장수 추가
* 동 단위 데이터가 없어 구단위 데이터로 반영

In [91]:
merged_df3['date'] = merged_df3['date_cd'].apply(lambda x: datetime.strptime(x, '%Y%m%d').strftime('%Y-%m-%d'))

In [92]:
construct_cnt_df = pd.DataFrame()

for idx, dt in enumerate(merged_df3['date'].unique()):
    temp = construct_df[(construct_df['cons_start_dt'] <= dt)
                        & (construct_df['cons_end_dt'] > dt)]
    out = pd.DataFrame(temp.groupby('gu')['cons_start_dt'].count()).reset_index()
    out['date'] = dt
    out.columns = ['gu', 'cons_cnt', 'date']
    if idx == 0:
        construct_cnt_df = out
    else:
        construct_cnt_df = pd.concat([construct_cnt_df, out], axis=0)

## 최종 데이터셋 생성

In [93]:
df = pd.merge(
        merged_df3,
        construct_cnt_df,
        how='left',
        on=['date', 'gu']
    )

In [94]:
df.head()

,date_cd,gu,dong,temperature,humidity,avg_re_dust,CRTR_DT,dong2_x,BUS_PSGR_CNT,dong2_y,...,MSRRGN_NM,MSRSTE_NM,PM10,PM25,O3,NO2,CO,SO2,date,cons_cnt
0,20220207,용산구,이촌동,1.0,36.0,46.0,20220207,이촌동,7290,이촌동,...,도심권,용산구,28,16,0.021,0.031,0.5,0.004,2022-02-07,1
1,20220311,용산구,이촌동,11.0,63.0,13.0,20220311,이촌동,8118,이촌동,...,도심권,용산구,67,36,0.022,0.043,0.6,0.004,2022-03-11,1
2,20220318,용산구,이촌동,9.0,67.0,20.0,20220318,이촌동,7655,이촌동,...,도심권,용산구,6,3,0.032,0.010,0.3,0.002,2022-03-18,1
3,20220518,용산구,이촌동,26.0,46.0,139.0,20220518,이촌동,9342,이촌동,...,도심권,용산구,59,34,0.079,0.017,0.5,0.004,2022-05-18,3
4,20220518,용산구,서빙고동,26.0,45.0,16.0,20220518,서빙고동,7282,서빙고동,...,도심권,용산구,59,34,0.079,0.017,0.5,0.004,2022-05-18,3


In [95]:
df = df[[
    'date_cd', 'dong', 'temperature', 'humidity', 'avg_re_dust', 
    'PM10', 'PM25', 'O3', 'NO2', 'CO', 'SO2', 'BUS_PSGR_CNT', '빈도수', 'cons_cnt'
]]

In [96]:
df.columns = ['date_cd' ,'dong', 'temp', 'hum', 'redust', 'pm10', 'pm25', 'o3', 'no2', 'co', 'so2', 'buspop', 'stops', 'consts']

In [97]:
df.to_csv('../data/main_df2.csv')

In [99]:
df.describe()

,temp,hum,redust,pm10,pm25,o3,no2,co,so2,buspop,stops,consts
count,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000
mean,19.358772,57.562281,37.723684,33.909649,18.307018,0.031239,0.021105,0.427719,0.002868,26626.779825,64.423684,29.487719
std,10.194606,17.879721,67.800526,21.257518,11.587304,0.013748,0.010256,0.146616,0.000704,17554.763650,47.810079,16.096418
min,-5.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2969.000000,4.000000,1.000000
25%,12.000000,44.000000,11.000000,22.000000,10.000000,0.021000,0.014000,0.300000,0.002000,13179.500000,26.000000,18.000000
50%,22.000000,58.000000,19.000000,29.000000,15.000000,0.030000,0.018000,0.400000,0.003000,21479.000000,51.000000,28.000000
75%,28.000000,70.000000,39.000000,40.000000,24.000000,0.040000,0.026250,0.500000,0.003000,36786.750000,97.000000,38.000000
max,35.000000,100.000000,1310.000000,213.000000,70.000000,0.079000,0.060000,1.000000,0.006000,82916.000000,190.000000,106.000000
